# RGB image compression

Compressing color images with PNG in the RGB domain.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
#!pwd
!ln -sf ~/MRVC/src/deadzone.py .
!ln -sf ~/MRVC/src/frame.py .
!ln -sf ~/MRVC/src/distortion.py .
!ln -sf ~/MRVC/src/information.py
!ln -sf ~/MRVC/src/debug.py
!ln -sf ~/MRVC/src/image_3.py
!ln -sf ~/MRVC/src/image_1.py
import deadzone as Q
import distortion
import image_3 as color_frame
import image_1 as gray_frame
#import rate

## Configuration area

In [ ]:
# Prefix of the RGB image to be quantized.

home = os.environ["HOME"]
#fn = home + "/MRVC/sequences/stockholm/"
fn = home + "/MRVC/sequences/lena_color/"

# Number of quantization steps.
Q_steps = 8

components = ['R', 'G', 'B']

## Generation of the optimal RD curve

The optimal RD curve can be generated with:

1. The RD curve of each RGB channel is computed, for a number of quantization steps.
2. The OTP (Optimal Truncation Points) are sorted by slopes.

In [ ]:
# Dead-zone quantizer & dequantizer.
# 
# Notice that, although this is a dead-zone quantizer, we are not going
# to work with negative samples, and therefore, the dead-zone
# does not have any effect.

def q_deq(x, quantization_step):
    k = Q.quantize(x, quantization_step)
    y = Q.dequantize(k, quantization_step)
    return k, y

In [ ]:
# Quantization indexes are stored in disk an normal pixels.

def _load_indexes(prefix):
    gray_frame.read(prefix)
    
def _write_indexes(prefix):
    gray_frame.write(prefix)

In [ ]:
# Read the image and show it.

img = color_frame.read(fn)
print(img.max(), img.min())

def show(img, title):
    img = color_frame.normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(img)

show(img, fn + "000.png")

In [ ]:
# Some helper functions.

# Number of bytes that a frame "img" requires in disk.
def bytes_per_color_frame(img):
    color_frame.write(img, "/tmp/frame")
    length_in_bytes = os.path.getsize("/tmp/frame000.png")
    return length_in_bytes

# The same value, but in bits/pixel.
def bits_per_color_pixel(img):
    return 8*bytes_per_color_frame(img)/(img.shape[0]*img.shape[1])

# Specific version for grayscale images.
def bytes_per_gray_frame(img):
    cv2.imwrite("/tmp/frame000.png", img)
    length_in_bytes = os.path.getsize("/tmp/frame000.png")
    return length_in_bytes

# The same value, but in bits/pixel.
def bits_per_gray_pixel(img):
    return 8*bytes_per_gray_frame(img)/(img.shape[0]*img.shape[1])

# Entropy of a sequence of symbols (pixels). This is a theoretical measure
# of the compression ratio of the sequence.
def entropy_in_bits_per_symbol(sequence_of_symbols):
    value, counts = np.unique(sequence_of_symbols, return_counts = True)
    probs = counts / len(sequence_of_symbols)
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    entropy = 0.
    for i in probs:
        entropy -= i * math.log(i, 2)

    return entropy

In [ ]:
def R_RD_curve(RGB_frame, Q_steps):
    RD_points = []
    for q in range(Q_steps-1, -1, -1):
        R_frame = RGB_frame[:,:,0]
        q_step = 1 << q
        k, dequantized_R_frame = q_deq(R_frame, q_step)
        k = k.astype(np.uint8)
        rate = bits_per_gray_pixel(k)
        _distortion = distortion.MSE(R_frame, dequantized_R_frame)
        RD_points.append((rate, _distortion, 'R', q_step))
        print(f"q_step={q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def G_RD_curve(RGB_frame, Q_steps):
    RD_points = []
    for q in range(Q_steps-1, -1, -1):
        G_frame = RGB_frame[:,:,1]
        q_step = 1 << q
        k, dequantized_G_frame = q_deq(G_frame, q_step)
        k = k.astype(np.uint8)
        rate = bits_per_gray_pixel(k)
        _distortion = distortion.MSE(G_frame, dequantized_G_frame)
        RD_points.append((rate, _distortion, 'G', q_step))
        print(f"q_step={q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def B_RD_curve(RGB_frame, Q_steps):
    RD_points = []
    for q in range(Q_steps-1, -1, -1):
        B_frame = RGB_frame[:,:,2]
        q_step = 1 << q
        k, dequantized_B_frame = q_deq(B_frame, q_step)
        k = k.astype(np.uint8)
        rate = bits_per_gray_pixel(k)
        _distortion = distortion.MSE(B_frame, dequantized_B_frame)
        RD_points.append((rate, _distortion, 'B', q_step))
        print(f"q_step={q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

R_points = R_RD_curve(img, Q_steps)
G_points = G_RD_curve(img, Q_steps)
B_points = B_RD_curve(img, Q_steps)

In [ ]:
R_points

In [ ]:
G_points

In [ ]:
B_points

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*[(i[0], i[1]) for i in R_points]), c='r', marker="o",
           label='R')              
pylab.plot(*zip(*[(i[0], i[1]) for i in G_points]), c='g', marker="o",
           label='G')              
pylab.plot(*zip(*[(i[0], i[1]) for i in B_points]), c='b', marker="o",
           label='B')              
pylab.title("RD Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

## Compute the slopes
In a different list for each component.

In [ ]:
def compute_slopes(RD_points):
    extended_RD_points = [(0.0, 0.0, '', -1)] + RD_points
    counter = 0
    RD_slopes = [(9.0E9, RD_points[0])]
    points_iterator = iter(RD_points)
    next(points_iterator)
    for i in points_iterator:
        BPP = i[0] # Rate 
        #print(RD_points[counter])
        #delta_BPP = BPP - extended_RD_points[counter][0]
        delta_BPP = BPP - RD_points[counter][0]
        MSE = i[1] # Distortion
        #delta_MSE = MSE - extended_RD_points[counter][1] 
        delta_MSE = MSE - RD_points[counter][1] 
        #print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
        if delta_BPP > 0:
            slope = abs(delta_MSE/delta_BPP)
        else:
            slope = 0
        component = i[2]
        q_step = i[3]
        print((slope, i), delta_MSE, delta_BPP)
        RD_slopes.append((slope, i))
        counter += 1
    #RD_slopes.append((0.0, (0.0, RD_points[-1][2], 1)))
    return RD_slopes

R_slopes = compute_slopes(R_points)
G_slopes = compute_slopes(G_points)
B_slopes = compute_slopes(B_points)

In [ ]:
R_slopes

In [ ]:
G_slopes

In [ ]:
B_slopes

## Merge the RD slopes and sort them
By slope.

In [ ]:
all_slopes = R_slopes + G_slopes + B_slopes
sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]

In [ ]:
sorted_slopes

## Build the optimal RD curve

Progressively quantize the image using the Q_steps described in the sorted list of monotonously decreasing slopes, and then, compute the distortion and the bit-rate. Notice that the distortion could have been deduced from the distortion in each component because the RGB domain is additive. However, the bit-rate is more difficult to be deduced from the bit-rate generated by each component, because the sum of the bit-rates of the components in general is slighly higher than considering the color image as a whole.

In [ ]:
def get_optimal_RD_curve(color_img, sorted_slopes, components):
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image.
    k = np.empty_like(color_img)
    y = np.empty_like(color_img)
    for i in sorted_slopes:
        point = i[1]
        component = point[2]
        Q_step = point[3]
        Q_steps_per_component[components.index(component)] = Q_step
        #print(i, Q_steps_per_component)
        for c,Q_step in zip(components, Q_steps_per_component):
            k[..., components.index(c)], y[..., components.index(c)] = q_deq(color_img[..., components.index(c)], Q_step)
        k = k.astype(np.uint8)
        rate = bits_per_color_pixel(k)
        _distortion = distortion.MSE(color_img, y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return points

optimal_points = get_optimal_RD_curve(img, sorted_slopes, components)

## RD curve using same $\Delta$ for each RGB channel ($\Delta_{\text{R}} = \Delta_{\text{G}} = \Delta_{\text{B}}$)
To see the contribution of each channel to the RD curve.

In [ ]:
def constant_Q_RD_curve(color_img, Q_steps):
    points = []
    for q in range(Q_steps-1, -1, -1):
        Q_step = 1 << q
        k, y = q_deq(color_img, Q_step)
        k = k.astype(np.uint8)
        rate = bits_per_color_pixel(k)
        _distortion = distortion.MSE(color_img, y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return points

constant_Q_RD_points = constant_Q_RD_curve(img, Q_steps)

## Let's compare!

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*optimal_points), c='g', marker="o", label="Optimal")
pylab.plot(*zip(*constant_Q_RD_points), c='m', marker="x", label="$\Delta_{\mathrm{R}}=\Delta_{\mathrm{G}}=\Delta_{\mathrm{B}}}$")
pylab.title("Rate/Distortion Performance ")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
pylab.legend(loc='upper right')
pylab.show()

## Conclusion

In the RGB domain, the optimal RD curve matches the constant quantization RD curve because the 3 components have the same weight in the contribution to the quality the reconstruction. However, we have more OTPs (Optimal Truncation Points) in the optimal one.

## Ignore the rest ...

In [ ]:
def accumulate_rate(points_sorted_by_slopes):
    optimal_points = []
    accumulated_BR = 0.0
    for i in points_sorted_by_slopes:
        BPP = i[1][0]; MSE = i[1][1]
        accumulated_BR += BPP
        optimal_points.append((accumulated_BR, MSE))
    return optimal_points

optimal_points = accumulate_rate(optimal_slopes)

In [ ]:
optimal_points

## Compute slopes

In [ ]:
def compute_slopes(RD_points):
    extended_RD_points = [(0.0, 9.0E9, '', -1)] + RD_points
    counter = 0
    RD_slopes = []
    for i in RD_points:
        BPP = i[0] # Rate 
        print(RD_points[counter])
        delta_BPP = BPP - extended_RD_points[counter][0]
        MSE = i[1] # Distortion
        delta_MSE = MSE - extended_RD_points[counter][1] 
        #print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
        if delta_BPP > 0:
            slope = delta_MSE/delta_BPP
        else:
            slope = 0
        RD_slopes.append((slope, i[2], i[3]))
        counter += 1
    return RD_slopes

RD_slopes = compute_slopes(sorted_RD_points)

In [ ]:
RD_slopes

In [ ]:
print(RD_points)

In [ ]:
with open('RGB.txt', 'w') as f:
    for item in RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

In [ ]:
k, y = q_deq(img, 64)
show(y, "")

## RD curves of each channel

In [ ]:
def only_R_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        R_frame = RGB_frame[:,:,0]
        k, dequantized_R_frame = q_deq(R_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bits_per_graypixel(k)
        _distortion = distortion.MSE(R_frame, dequantized_R_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def only_G_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        G_frame = RGB_frame[:,:,1]
        k, dequantized_G_frame = q_deq(G_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bits_per_graypixel(k)
        _distortion = distortion.MSE(G_frame, dequantized_G_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def only_B_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        B_frame = RGB_frame[:,:,2]
        k, dequantized_B_frame = q_deq(B_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bits_per_graypixel(k)
        _distortion = distortion.MSE(B_frame, dequantized_B_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

only_R_points = only_R_RD_curve(img)
only_G_points = only_G_RD_curve(img)
only_B_points = only_B_RD_curve(img)

In [ ]:
only_B_points

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker="x",
           label='$\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$')
pylab.plot(*zip(*only_R_points), c='r', marker="o",
           label='Only R')              
pylab.plot(*zip(*only_G_points), c='g', marker="o",
           label='Only G')              
pylab.plot(*zip(*only_B_points), c='b', marker="o",
           label='Only B')              
pylab.title("RD Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

The $\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$ quantization scheme is near optimal because the slope at the different quantization points is almost the same. This can be seen in the next experiment

## Testing a different quantization configuration

In [ ]:
N = 6
def only_R_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        dequantized_RGB_frame = np.empty_like(RGB_frame)
        k = np.empty_like(RGB_frame)
        k[:,:,0], dequantized_RGB_frame[:,:,0] = q_deq(RGB_frame[:,:,0], 1<<q_step)
        k[:,:,1], dequantized_RGB_frame[:,:,1] = q_deq(RGB_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_RGB_frame[:,:,2] = q_deq(RGB_frame[:,:,2], 1<<N)
        k = k.astype(np.uint8)
        rate = bits_per_pixel(k)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def only_G_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        dequantized_RGB_frame = np.empty_like(RGB_frame)
        k = np.empty_like(RGB_frame)
        k[:,:,0], dequantized_RGB_frame[:,:,0] = q_deq(RGB_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_RGB_frame[:,:,1] = q_deq(RGB_frame[:,:,1], 1<<q_step)
        k[:,:,2], dequantized_RGB_frame[:,:,2] = q_deq(RGB_frame[:,:,2], 1<<N)
        k = k.astype(np.uint8)
        rate = bits_per_pixel(k)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

def only_B_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        dequantized_RGB_frame = np.empty_like(RGB_frame)
        k = np.empty_like(RGB_frame)
        k[:,:,0], dequantized_RGB_frame[:,:,0] = q_deq(RGB_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_RGB_frame[:,:,1] = q_deq(RGB_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_RGB_frame[:,:,2] = q_deq(RGB_frame[:,:,2], 1<<q_step)
        k = k.astype(np.uint8)
        rate = bits_per_pixel(k)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
    return RD_points

only_R_points = only_R_RD_curve(img)
only_G_points = only_G_RD_curve(img)
only_B_points = only_B_RD_curve(img)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker="x",
           label='$\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$')
pylab.plot(*zip(*only_R_points), c='r', marker="o",
           label='$\Delta_{\mathrm{R}}~\mathrm{varies},~\Delta_{\mathrm{G}}=\Delta_{\mathrm{B}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_G_points), c='g', marker="o",
           label='$\Delta_{\mathrm{G}}~\mathrm{varies},~\Delta_{\mathrm{R}}=\Delta_{\mathrm{B}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_B_points), c='b', marker="o",
           label='$\Delta_{\mathrm{B}}~\mathrm{varies},~\Delta_{\mathrm{R}}=\Delta_{\mathrm{G}}=$' + '{}'.format(1<<N))              
pylab.title("RD Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

As it can be seen, the best configuration matches $\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$.

Lo que hay que hacer es calcular el slope para cada OTP (Optimal Truncated Point) de cada canal, ordenarlos y trazar la curva RD. Esto nos daría la curva RD óptima.